In [144]:
import os
import sys
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,count,isnan,when,sum,lit


In [145]:
def init_spark():
    spark = SparkSession.builder.appName("Python Spark SQL basic example").config("spark.some.config.option", "some-value").getOrCreate()
    return spark

In [146]:
spark_object = init_spark()
train_identity = spark_object.read.option("delimiter", ",").csv("./data/train_identity.csv", header='true', inferSchema='true')
train_transaction = spark_object.read.option("delimiter", ",").csv("./data/train_transaction.csv", header='true', inferSchema='true')

In [147]:
print(train_identity.count())
print(train_transaction.count())
print(train_identity.head())
print(train_transaction.head())

144233
590540
Row(TransactionID=2987004, id_01=0.0, id_02=70787.0, id_03=None, id_04=None, id_05=None, id_06=None, id_07=None, id_08=None, id_09=None, id_10=None, id_11=100.0, id_12='NotFound', id_13=None, id_14=-480.0, id_15='New', id_16='NotFound', id_17=166.0, id_18=None, id_19=542.0, id_20=144.0, id_21=None, id_22=None, id_23=None, id_24=None, id_25=None, id_26=None, id_27=None, id_28='New', id_29='NotFound', id_30='Android 7.0', id_31='samsung browser 6.2', id_32=32.0, id_33='2220x1080', id_34='match_status:2', id_35='T', id_36='F', id_37='T', id_38='T', DeviceType='mobile', DeviceInfo='SAMSUNG SM-G892A Build/NRD90M')
Row(TransactionID=2987000, isFraud=0, TransactionDT=86400, TransactionAmt=68.5, ProductCD='W', card1=13926, card2=None, card3=150.0, card4='discover', card5=142.0, card6='credit', addr1=315.0, addr2=87.0, dist1=19.0, dist2=None, P_emaildomain=None, R_emaildomain=None, C1=1.0, C2=1.0, C3=0.0, C4=0.0, C5=0.0, C6=1.0, C7=0.0, C8=0.0, C9=1.0, C10=0.0, C11=2.0, C12=0.0, C

In [148]:
train = train_transaction.join(train_identity, on="TransactionID", how='left')

In [149]:
test_identity = spark_object.read.option("delimiter", ",").csv("./data/test_identity.csv", header='true', inferSchema='true')
test_transaction = spark_object.read.option("delimiter", ",").csv("./data/test_transaction.csv", header='true', inferSchema='true')

In [150]:
#print(train.count())
#train.head()

In [151]:
# total_count = train.count()
# def null_percentage(df, column):
#     null_count = df.where(col(column).isNull()).count()
#     return null_count / total_count

 
# for column in train.columns:
#     if null_percentage(train, column) > 0.05:
#         train = train.drop(col(column))

In [153]:
# As some ids are incorrectly labelled, we will correct it first
id_cols = [col for col in test_identity.columns if col.startswith('id')]
rename_cols = {col: 'id_' + col[-2:] for col in id_cols}
test_identity = test_identity.select([col(c).alias(rename_cols.get(c, c)) for c in test_identity.columns])
# join test_transaction and test_identity on TransactionID
test = test_transaction.join(test_identity,on="TransactionID",how='left')
train.write.csv('./data/train_combined.csv',header = True)
test.write.csv('./data/test_combined.csv',header = True)

In [155]:
train_combined = spark_object.read.option("delimiter", ",").csv("data/train_combined.csv", header='true', inferSchema='true')
test_combined = spark_object.read.option("delimiter", ",").csv("data/test_combined.csv", header='true', inferSchema='true')

print((train_combined.count(),len(train_combined.columns)))
print((test_combined.count(),len(test_combined.columns)))

(590540, 434)
(506691, 433)


In [156]:
def top_missing_cols(df, n=None, thresh=80):
    """
    returns missing columns in dataframe with missing values percent > thresh
    if n=None. It will gave whole dataframe with missing values percent > thresh
    """

    # Get the percentage of missing values for each column
    dff = df.select([(count(when(isnan(c) | col(c).isNull(), c))/count('*'))*100 for c in df.columns])
    dff = dff.toDF(*df.columns)

    # Count the total number of columns in the DataFrame
    total_cols = len(df.columns)

    # Convert the DataFrame to Pandas and process it
    dff = dff.toPandas()
    dff = dff.melt(var_name='col', value_name='missing_percent')
    dff = dff.sort_values(by=['missing_percent'], ascending=False).reset_index(drop=True)

    print(f'There are {total_cols} columns in this dataset.')
    print(f'There are {dff[dff["missing_percent"] > thresh].shape[0]} columns with missing percent values than {thresh}%')
    
    # Return the top missing columns
    if n:
        return dff.head(n)
    else:
        return dff

# Get the columns with missing percentage greater than 50%




In [157]:

df_missing = top_missing_cols(train_combined, n=None, thresh=50)
missing_cols = df_missing['col']

C:\Program Files\Apache Spark\spark-3.1.2-bin-hadoop2.7\spark-3.1.2-bin-hadoop2.7\python\pyspark\sql\pandas\conversion.py:186: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
C:\Program Files\Apache Spark\spark-3.1.2-bin-hadoop2.7\spark-3.1.2-bin-hadoop2.7\python\pyspark\sql\pandas\conversion.py:186: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
C:\Program Files\Apache Spark\spark-3.1.2-bin-hadoop2.7\spark-3.1.2-bin-hadoop2.7\python\pyspark\sql\pandas\conversion.py:186: Performance

There are 434 columns in this dataset.
There are 214 columns with missing percent values than 50%


In [158]:
nan_dict = {}
for col_name in missing_cols:
    count = train_combined.select([count(when(col(c).isNull(), c)).alias(c) for c in [col_name]]).collect()[0][0]
    try:
        nan_dict[count].append(col_name)
    except:
        nan_dict[count] = [col_name]

for k,v in nan_dict.items():
    print(f'#####' * 4)
    print(f'NAN count = {k} percent: {(int(k)/train.count())*100} %')
    print(v)

TypeError: 'int' object is not callable